# POST Requests with Python

So far we've worked with **GET** requests. **POST** requests are just as common and very useful when we want to **send data to a server** and actually **cause a change of state** on the server or on a remote application.

- Use **`data`** for **form-encoded** request bodies.
- Use **`json`** for **JSON** payloads.

When we pass a Python dictionary to the **`json`** keyword argument, the `requests` library will:
1. **Serialize** the dictionary to JSON automatically—we don't need to call `json.dumps()` ourselves.
2. **Set the `Content-Type`** header to `application/json` automatically, so we don't need to set that header explicitly.

We can inspect the response the same way as with GET: **`response.status_code`**, **`response.json()`** to parse the body, and **`response.raise_for_status()`** to raise an exception on 4xx/5xx and get a fail-fast approach.

## Why httpbin.org instead of GitHub?

For **POST** requests against the GitHub API, we would need **authentication** set up for most endpoints. POST requests perform operations on the server and on data, so authentication is typically required.

For a simple, **no-auth** example we use **[httpbin.org](https://httpbin.org)**—a service that lets you simulate different HTTP methods, authentication, status codes, and more. It **echoes back** what you send, so we can introspect how the `requests` library builds the request (headers, body, etc.).

In [2]:
import requests
import json

POST_ECHO_URL = "https://httpbin.org/post"

## Sending a JSON payload

We define a **payload** as a Python dictionary. We pass it to **`requests.post(..., json=payload)`**. The library serializes it to JSON and sets `Content-Type: application/json`. We use **`timeout=10`** to avoid hanging and **`raise_for_status()`** to fail fast on errors.

In [3]:
payload = {
    "script_name": "DevOps automation",
    "action": "trigger_deployment",
    "environment": "staging",
    "version": "1.0.5",
}

response = requests.post(POST_ECHO_URL, json=payload, timeout=10)
response.raise_for_status()

print(json.dumps(response.json(), indent=2))

{
  "args": {},
  "data": "{\"script_name\": \"DevOps automation\", \"action\": \"trigger_deployment\", \"environment\": \"staging\", \"version\": \"1.0.5\"}",
  "files": {},
  "form": {},
  "headers": {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate",
    "Content-Length": "114",
    "Content-Type": "application/json",
    "Host": "httpbin.org",
    "User-Agent": "python-requests/2.32.5",
    "X-Amzn-Trace-Id": "Root=1-6995841a-21d90e5e723fb726566884f7"
  },
  "json": {
    "action": "trigger_deployment",
    "environment": "staging",
    "script_name": "DevOps automation",
    "version": "1.0.5"
  },
  "origin": "62.90.240.225",
  "url": "https://httpbin.org/post"
}


### What we get back

The httpbin response includes:
- **`headers`** – the headers that were sent (so we can see that `Content-Type` was set to `application/json`).
- **`json`** – the original payload we sent (echoed back).

This schema is specific to httpbin; other APIs will return their own format. The echo behavior is useful for learning and debugging what the `requests` library actually sends.

## Common pitfalls

| Pitfall | Why it matters |
|--------|-----------------|
| **Not setting timeouts** | Scripts can hang indefinitely if the server or network is slow. Always include a `timeout` (e.g. `timeout=10`) in your requests. |
| **Ignoring HTTP errors** | Without `raise_for_status()`, 4xx and 5xx responses don't raise an exception. You can get silent failures. Use **fail-fast**: call `raise_for_status()` and handle errors explicitly. |
| **Using `data` instead of `json`** | `data=` sends **form-encoded** data. Many modern APIs expect **JSON**. Use **`json=`** for JSON payloads so the body and `Content-Type` are correct. |
| **Hard-coding secrets** | API keys and tokens are sensitive. **Never** hard-code them. Use **environment variables**; for local development, use a **`.env`** file (e.g. with `python-dotenv`). For production, use a proper **secrets management** solution (e.g. HashiCorp Vault, AWS Secrets Manager). |

In [4]:
# Good practices in one place:
# response = requests.post(url, json=payload, timeout=10)
# response.raise_for_status()
# result = response.json()